In [3]:
import face_recognition
import cv2
import numpy as np
import time

In [9]:
# Get a reference to webcam #0 (the default one)
video_capture = cv2.VideoCapture(0)

# Create arrays of known face encodings and their names
known_face_encodings = []
known_face_names = []
appeared_time = []
counts = []
count = 0
# Initialize some variables
face_locations = []
face_encodings = []
face_names = []
process_this_frame = True

while True:
    # Grab a single frame of video
    ret, frame = video_capture.read()
    if not ret:
        break
        
    # Resize frame of video to 1/4 size for faster face recognition processing
    small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)

    # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
    rgb_small_frame = small_frame[:, :, ::-1]

    # Only process every other frame of video to save time
    if process_this_frame:
        # Find all the faces and face encodings in the current frame of video
        #Remove model = cnn if you want to run on cpu
        face_locations = face_recognition.face_locations(rgb_small_frame)
        face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

        face_names = []
        face_count = []
        for face_encoding in face_encodings:
            # See if the face is a match for the known face(s)
            matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
            
            if (len(matches)==0) or (not(True in matches)):
                known_face_encodings.append(face_encoding)
                known_face_names.append('person'+str(count))
                appeared_time.append(round(time.time(),2))
                counts.append(1)
                count+=1

            # If a match was found in known_face_encodings, just use the first one.
            if True in matches:
                first_match_index = matches.index(True)
                name = known_face_names[first_match_index]
                last_time = appeared_time[first_match_index]
                appeared_time[first_match_index] = round(time.time(),2)
                if abs(appeared_time[first_match_index]-last_time)>1:
                    counts[first_match_index]+=1
                face_names.append(name)
                face_count.append(counts[first_match_index])


    process_this_frame = not process_this_frame

    cv2.putText(frame,"People:{}".format(count),(10,30),cv2.FONT_HERSHEY_SIMPLEX,0.7,(0,0,255),2)
    # Display the results
    for (top, right, bottom, left), appear, name in zip(face_locations, face_count, face_names):
        # Scale back up face locations since the frame we detected in was scaled to 1/4 size
        top *= 4
        right *= 4
        bottom *= 4
        left *= 4

        # Draw a box around the face
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

        # Draw a label with a name below the face
        cv2.rectangle(frame, (left, bottom - 35), (right, bottom+20), (0, 0, 255), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, "{0}".format(name), (left, bottom-10), font, 1.0, (255, 255, 255), 1)
        cv2.putText(frame, "count:{0}".format(appear), (left, bottom+15), font, 1.0, (255, 255, 255), 1)
#         cv2.putText(frame,"first apperead {} seconds ago".format(timing),(10,30),cv2.FONT_HERSHEY_SIMPLEX,0.7,(0,0,255),2)
    # Display the resulting image
    cv2.imshow('Video', frame)

    # Hit 'q' on the keyboard to quit!
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release handle to the webcam
video_capture.release()
cv2.destroyAllWindows()

In [7]:
video_capture.release()
cv2.destroyAllWindows()